In [1]:
import numpy as np
import pandas as pd
import random
import time
import sys
import os
from brian2 import *
import PIL 
from PIL import Image
import matplotlib.pyplot as plt
prefs.codegen.target = 'numpy'

In [2]:
trainData = pd.read_csv('mnist_train.csv', header=None)
x_train = (trainData.iloc[:,1:]).values
y_train = (trainData.iloc[:,0:1]).values

In [3]:
x_train.shape

(60000, 784)

In [4]:
testData = pd.read_csv('mnist_test.csv', header=None)
x_test = (testData.iloc[:,1:]).values
y_test = (testData.iloc[:,0:1]).values

In [5]:
neuron_eqs_e = '''
        dv/dt = ((v_rest_e - v) + (I_synE+I_synI) / nS) / (100*ms)  : volt
        I_synE = ge * nS *         -v                           : amp
        I_synI = gi * nS * (-100.*mV-v)                          : amp
        dge/dt = -ge/(1.0*ms)                                   : 1
        dgi/dt = -gi/(2.0*ms)                                  : 1
        '''

In [6]:
neuron_eqs_e += '\n  theta      :volt'

In [10]:
neuron_eqs_i = '''
        dv/dt = ((v_rest_i - v) + (I_synE+I_synI) / nS) / (10*ms)  : volt
        I_synE = ge * nS *         -v                           : amp
        I_synI = gi * nS * (-85.*mV-v)                          : amp
        dge/dt = -ge/(1.0*ms)                                   : 1
        dgi/dt = -gi/(2.0*ms)                                  : 1
        '''
eqs_stdp_ee = '''
                post2before                            : 1.0
                dpre/dt   =   -pre/(tc_pre_ee)         : 1.0
                dpost1/dt  = -post1/(tc_post_1_ee)     : 1.0
                dpost2/dt  = -post2/(tc_post_2_ee)     : 1.0
            '''
eqs_stdp_pre_ee = 'pre = 1.; w -= nu_ee_pre * post1'
eqs_stdp_post_ee = 'post2before = post2; w += nu_ee_post * pre * post2before; post1 = 1.; post2 = 1.'

In [15]:
n_input = 784
n_e = 400
n_i = n_e 

num_examples = 1000
population_names = ['A']

single_example_time =   350 * msecond #
resting_time = 150 * msecond
runtime = num_examples * (single_example_time + resting_time)

In [16]:
runtime

0.5 * ksecond

In [18]:
v_rest_e = -65. * mV 
v_rest_i = -60. * mV 
v_reset_e = -65. * mV
v_reset_i = -45. * mV
v_thresh_e = -52. * mV
v_thresh_i = -40. * mV
refrac_e = 5. * ms
refrac_i = 2. * ms

In [20]:
scr_e = 'v = v_reset_e; timer = 0*ms'

In [21]:
scr_e

'v = v_reset_e; timer = 0*ms'

In [25]:
neuron_groups = {}

In [26]:
neuron_groups['e'] = NeuronGroup(n_e*len(population_names), neuron_eqs_e, threshold= v_thresh_e, refractory= refrac_e, reset= scr_e)
neuron_groups['i'] = NeuronGroup(n_i*len(population_names), neuron_eqs_i, threshold= v_thresh_i, refractory= refrac_i, reset= v_reset_i)

In [31]:
ending = ''
connections = {}
recurrent_conn_names = ['ei', 'ie']

In [ ]:
def get_matrix_from_file(fileName):
    offset = len(ending) + 4
    if fileName[-4-offset] == 'X':
        n_src = n_input                
    else:
        if fileName[-3-offset]=='e':
            n_src = n_e
        else:
            n_src = n_i
    if fileName[-1-offset]=='e':
        n_tgt = n_e
    else:
        n_tgt = n_i
    readout = np.load(fileName)
    print readout.shape, fileName
    value_arr = np.zeros((n_src, n_tgt))
    if not readout.shape == (0,):
        value_arr[np.int32(readout[:,0]), np.int32(readout[:,1])] = readout[:,2]
    return value_arr

In [ ]:
for name in population_names:
    print 'create neuron group', name
    
    neuron_groups[name+'e'] = neuron_groups['e'].subgroup(n_e)
    neuron_groups[name+'i'] = neuron_groups['i'].subgroup(n_i)
    
    neuron_groups[name+'e'].v = v_rest_e - 40. * b.mV
    neuron_groups[name+'i'].v = v_rest_i - 40. * b.mV
    
    neuron_groups['e'].theta = np.load('C:/Users/云/stdp-mnist-master/weights/' + 'theta_' + name + ending + '.npy')

    
    print 'create recurrent connections'
    for conn_type in recurrent_conn_names:
        connName = name+conn_type[0]+name+conn_type[1]
        weightMatrix = get_matrix_from_file('C:/Users/云/stdp-mnist-master/' + 'random/' + connName + ending + '.npy')
        connections[connName] = Connection(neuron_groups[connName[0:2]], neuron_groups[connName[2:4]], structure= conn_structure, 
                                                    state = 'g'+conn_type[0])
        connections[connName].connect(neuron_groups[connName[0:2]], neuron_groups[connName[2:4]], weightMatrix)
                
    if ee_STDP_on:
        if 'ee' in recurrent_conn_names:
            stdp_methods[name+'e'+name+'e'] = b.STDP(connections[name+'e'+name+'e'], eqs=eqs_stdp_ee, pre = eqs_stdp_pre_ee, 
                                                           post = eqs_stdp_post_ee, wmin=0., wmax= wmax_ee)

    print 'create monitors for', name
    rate_monitors[name+'e'] = b.PopulationRateMonitor(neuron_groups[name+'e'], bin = (single_example_time+resting_time)/b.second)
    rate_monitors[name+'i'] = b.PopulationRateMonitor(neuron_groups[name+'i'], bin = (single_example_time+resting_time)/b.second)
    spike_counters[name+'e'] = b.SpikeCounter(neuron_groups[name+'e'])
    
    if record_spikes:
        spike_monitors[name+'e'] = b.SpikeMonitor(neuron_groups[name+'e'])
        spike_monitors[name+'i'] = b.SpikeMonitor(neuron_groups[name+'i'])

In [34]:
neuron_groups['e']

NeuronGroup(clock=Clock(dt=100. * usecond, name='defaultclock'), when=start, order=0, name='neurongroup_1')

In [33]:
name+'e'

'Ae'

In [60]:
name = population_names[0]

neuron_groups[name+'e'] = neuron_groups['e'][:400]
neuron_groups[name+'i'] = neuron_groups['i'][:400]

In [39]:
neuron_groups[name+'e'].v = v_rest_e - 40. * mV
neuron_groups[name+'i'].v = v_rest_i - 40. * mV

In [48]:
neuron_groups['e'].theta = np.load('C:/Users/云/stdp-mnist-master/weights/' + 'theta_' + name + ending + '.npy') * 1000 * mV

In [ ]:
conn_structure = 'dense'

In [ ]:
for conn_type in recurrent_conn_names:
    connName = name+conn_type[0]+name+conn_type[1]
    weightMatrix = get_matrix_from_file('C:/Users/云/stdp-mnist-master/' + 'random/' + connName + ending + '.npy')
    connections[connName] = Connection(neuron_groups[connName[0:2]], neuron_groups[connName[2:4]], state = 'g'+conn_type[0])
    connections[connName].connect(neuron_groups[connName[0:2]], neuron_groups[connName[2:4]], weightMatrix)       

1.connName = Aei
2.connName = Aie

In [65]:
'g'+recurrent_conn_names[0][0]

'ge'

In [63]:
recurrent_conn_names

['ei', 'ie']

In [ ]:
conn = Synapses(neuron_groups[connName[0:2]], neuron_groups[connName[2:4]], 'w : siemens', on_pre='ge += w')